<a href="https://colab.research.google.com/github/shwoa/Capstone/blob/main/CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.2 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving MutualInfo_SNP(7000).csv to MutualInfo_SNP(7000).csv
Saving 유전형인코딩_최빈값.csv to 유전형인코딩_최빈값.csv
Saving 토마토_표현형.CSV to 토마토_표현형.CSV


# **CATBOOST**

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings("ignore")

# 📁 파일 경로
snp_path = "MutualInfo_SNP(5000).csv"
geno_path = "유전형인코딩_최빈값.csv"
pheno_path = "2016년 표현형 데이터.xlsx"

# 📊 데이터 로딩
top_snp_df = pd.read_csv(snp_path)
geno_df = pd.read_csv(geno_path, index_col=0)
pheno_df = pd.read_excel(pheno_path, index_col=0)

# 🎯 예측 대상 표현형 목록
traits = top_snp_df["표현형"].unique()
results = []

for trait in traits:
    top_snps = top_snp_df[top_snp_df["표현형"] == trait]["SNP"].tolist()
    geno_top = geno_df.loc[top_snps].T  # 샘플 x SNP
    X = geno_top
    y = pheno_df[trait].loc[X.index]

    # 🔀 학습/검증 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 🧠 CatBoost 모델 학습
    model = CatBoostRegressor(verbose=0, random_seed=42)
    model.fit(X_train, y_train)

    # 📈 예측 및 성능 평가
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2  = r2_score(y_test, y_pred)
    results.append({"표현형": trait, "MSE": mse, "R²": r2})

# 📋 결과 출력
results_df = pd.DataFrame(results)
print(results_df)


         표현형          MSE        R²
0     과중 (g)  1027.569504  0.580651
1    과장 (mm)    46.278861  0.506283
2    과폭 (mm)    97.285010  0.610597
3  과피두께 (mm)     0.744144  0.473894
4     당도 (%)     0.487789  0.706528
5  과실경도 (kg)     0.008994  0.284075


In [ ]:
display(results_df)

,표현형,MSE,R²
0,과중 (g),1027.569504,0.580651
1,과장 (mm),46.278861,0.506283
2,과폭 (mm),97.285010,0.610597
3,과피두께 (mm),0.744144,0.473894
4,당도 (%),0.487789,0.706528
5,과실경도 (kg),0.008994,0.284075


# **웹사이트에 사용할 cbm 파일도 포함**

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import warnings, os
warnings.filterwarnings("ignore")

# 📁 파일 경로
snp_path = "MutualInfo_SNP(5000).csv"
geno_path = "유전형인코딩_최빈값.csv"
pheno_path = "2016년 표현형 데이터.xlsx"

# 📊 데이터 로딩
top_snp_df = pd.read_csv(snp_path)
geno_df = pd.read_csv(geno_path, index_col=0)
pheno_df = pd.read_excel(pheno_path, index_col=0)

# 🎯 예측 대상 표현형 목록
traits = top_snp_df["표현형"].unique()
results = []

# 🔧 모델 저장 폴더
os.makedirs("models", exist_ok=True)

for trait in traits:
    top_snps = top_snp_df[top_snp_df["표현형"] == trait]["SNP"].tolist()
    geno_top = geno_df.loc[top_snps].T  # 샘플 x SNP
    X = geno_top
    y = pheno_df[trait].loc[X.index]

    # 🔀 학습/검증 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 🧠 모델 학습
    model = CatBoostRegressor(verbose=0, random_seed=42)
    model.fit(X_train, y_train)

    # 💾 모델 저장 (.cbm)
    model.save_model(f"models/{trait}_catboost_model.cbm")

    # 📈 예측 및 성능 평가
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2  = r2_score(y_test, y_pred)
    results.append({"표현형": trait, "MSE": mse, "R²": r2})

# 📋 결과 출력
results_df = pd.DataFrame(results)
# print(results_df)
display(results_df)

,표현형,MSE,R²
0,과중 (g),1027.569504,0.580651
1,과장 (mm),46.278861,0.506283
2,과폭 (mm),97.285010,0.610597
3,과피두께 (mm),0.744144,0.473894
4,당도 (%),0.487789,0.706528
5,과실경도 (kg),0.008994,0.284075


In [ ]:
traits = ["과중 (g)", "과장 (mm)", "과폭 (mm)", "과피두께 (mm)", "과실경도 (kg)", "당도 (%)"]

for trait in traits:
    try:
        model = CatBoostRegressor()
        model.load_model(f"models/{trait}_catboost_model.cbm")
        print(f"✅ {trait} 모델 로딩 성공")
    except Exception as e:
        print(f"❌ {trait} 모델 로딩 실패:", e)


✅ 과중 (g) 모델 로딩 성공
✅ 과장 (mm) 모델 로딩 성공
✅ 과폭 (mm) 모델 로딩 성공
✅ 과피두께 (mm) 모델 로딩 성공
✅ 과실경도 (kg) 모델 로딩 성공
✅ 당도 (%) 모델 로딩 성공


In [ ]:
# 필요 데이터 로딩 (기존과 동일하게)
geno_df = pd.read_csv("유전형인코딩_최빈값.csv", index_col=0)
top_snp_df = pd.read_csv("MutualInfo_SNP(5000).csv")

# 예시 표현형
trait = "과중 (g)"

# 해당 표현형의 상위 SNP 추출
top_snps = top_snp_df[top_snp_df["표현형"] == trait]["SNP"].tolist()
geno_top = geno_df.loc[top_snps].T  # 샘플 x SNP

# 모델 불러오기
model = CatBoostRegressor()
model.load_model(f"models/{trait}_catboost_model.cbm")

# 예측
y_pred = model.predict(geno_top)
print(f"예측 완료 ✅ → 예측값 샘플: {y_pred[:5]}")


예측 완료 ✅ → 예측값 샘플: [86.37010676 32.46452299 43.46540257 14.62009622 44.90019408]


웹사이트 예시 유전형 파일 추출

In [ ]:
import pandas as pd
from catboost import CatBoostRegressor
import os

# ✅ .cbm 파일 목록 (동일 폴더에 위치해야 함)
model_files = [
    "과중 (g)_catboost_model.cbm",
    "과장 (mm)_catboost_model.cbm",
    "과폭 (mm)_catboost_model.cbm",
    "과피두께 (mm)_catboost_model.cbm",
    "과실경도 (kg)_catboost_model.cbm",
    "당도 (%)_catboost_model.cbm",
]

# ✅ 모든 모델에서 feature(SNP) 이름 수집
all_snps = set()

for model_path in model_files:
    if os.path.exists(model_path):
        model = CatBoostRegressor()
        model.load_model(model_path)
        all_snps.update(model.feature_names_)
    else:
        print(f"⚠️ 파일 없음: {model_path}")

# ✅ 정렬된 리스트로 변환
snp_list = sorted(list(all_snps))

# ✅ 예시 데이터 생성 (샘플 3개, 고정된 SNP 값 0/1/2 반복)
sample_ids = ["S001", "S002", "S003"]
data = []

for sample in sample_ids:
    row = {"SampleID": sample}
    for i, snp in enumerate(snp_list):
        row[snp] = [0, 1, 2][i % 3]  # 0, 1, 2 반복
    data.append(row)

# ✅ DataFrame 생성
df = pd.DataFrame(data)
df = df.set_index("SampleID")

# ✅ 저장
df.to_csv("예시_전체SNP_유전형.csv", encoding="utf-8-sig")
print("✅ 예시 유전형 CSV 저장 완료: 예시_전체SNP_유전형.csv")

✅ 예시 유전형 CSV 저장 완료: 예시_전체SNP_유전형.csv
